# Importing Libraries

In [1]:
import numpy as np, pandas as pd

!pip install talos

import re
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import string
from string import ascii_lowercase

from tqdm import tqdm_notebook
import itertools
import io

import matplotlib.pyplot as plt
%matplotlib inline

from functools import reduce
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from keras import initializers, regularizers, constraints, optimizers, layers
import talos

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aaryan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Importing Data

In [5]:
#from google.colab import files
 
 
#uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [4]:
train=pd.read_csv('jigsaw-toxic-comment-train.csv')

In [5]:
#uploadedtest = files.upload()
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [6]:
test=pd.read_csv('test.csv')

# Data Exploration

Checking for missing values

In [7]:
train.isnull().any()

id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

In [8]:
test.isnull().any()


id         False
content    False
lang       False
dtype: bool

In [9]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
y = train[labels].values

#Data Pre-processing

## Text Normalization

* Removing Characters in between Text
* Removing Repeated Characters
* Converting data to lower-case
* Removing Numbers from the data
* Remove Punctuation
* Remove Whitespaces
* Removing spaces in between words
* Removing "\n"
* Remove Non-english characters

In [10]:
RE_PATTERNS = {
    ' american ':
        [
            'amerikan'
        ],

    ' adolf ':
        [
            'adolf'
        ],


    ' hitler ':
        [
            'hitler'
        ],

    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*', 'f**k','fu*k',
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'
        ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$',
            '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s','a55', '@$$'
        ],

    ' ass hole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole', 'a**hole'
        ],

    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h', 'b!tch', 'bi+ch', 'l3itch'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],

    ' trans gender':
        [
            'transgender'
        ],

    ' gay ':
        [
            'gay'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k', 'dik'
        ],

    ' suck ':
        [
            'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bull shit ':
        [
            'bullsh\*t', 'bull\$hit'
        ],

    ' homo sex ual':
        [
            'homosexual'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots'
                                                                                      'i d i o t'
        ],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t', '$h1t'
        ],

    ' shit hole ':
        [
            'shythole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            ' raped'
        ],

    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],

    ' ass head':
        [
            'butthead'
        ],

    ' sex ':
        [
            'sexy', 's3x', 'sexuality'
        ],


    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],

    ' shut the fuck up':
        [
            'stfu', 'st*u'
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses', 'p*ssy'
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' mother fucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker',
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],
    ' fucking ':
        [
            'f*$%-ing'
        ],
}


In [11]:
#Ascii Removal step
def asciiremoval(string):
    string = re.sub('[^a-zA-Z.\d\s]', '', string)
    return string

#Removing dates
def remove_dates(sentence):
    for i in range(0,277):
        df1[i] = df1[i].sub('(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{2}\s\d{4}', ' ', sentence)

#Removing Url
def remove_url(txt):
    txt_nourl=re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', txt)
    return txt_nourl

#Punctuation Removal
def removepunct(txt):
    txt_nopunct="".join((c for c in txt if c not in string.punctuation))
    return txt_nopunct

contractions = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}










In [12]:
def clean_text(text,remove_repeat_text=True, remove_patterns_text=True, is_lower=True):

  if is_lower:
    text=text.lower()
    
  if remove_patterns_text:
    for target, patterns in RE_PATTERNS.items():
      for pat in patterns:
        text=str(text).replace(pat, target)
    

  if remove_repeat_text:
    text = re.sub(r'(.)\1{2,}', r'\1', text) 

  text = str(text).replace("\n", " ")
#Ascii Removal
  text = re.sub('[^a-zA-Z.\d\s]', '', text)
#URL removal
  text=re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', text)
#Remove all the special characters 
  text = re.sub(r'[^\w\s]',' ',text)
  text = re.sub('[0-9]',"",text)
  text = re.sub(" +", " ", text)
#Date Removal 
  text=re.sub('(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{2}\s\d{4}', ' ', text)
  text= re.sub('[^a-zA-Z.\d\s]', '', text)
  text = re.sub("([^\x00-\x7F])+"," ",text)
  return text 





Cleaning Training Data

In [13]:
train['comment_text']=train['comment_text'].apply(lambda x: clean_text(x))
train['comment_text'][1]

'daww he matches this background colour im seemingly stuck with thanks talk january utc'

Cleaning Test Data

In [14]:
test['content']=test['content'].apply(lambda x: clean_text(x))
test['content'][1048]

'franchement c est fastidieux de perdre du temps avec un contributeur qui visibelement joue au con avec moi depuis des mois en se livrant un pov pushing pourtant d avance vou l chec face des sources unanimes je ne peux plus supposer la bonne foi dans ces conditions merci d arrter de nous faire perdre du temps apollofox discuter '

In [15]:
train.to_csv('Data/train_preprocessed_first.csv')
test.to_csv('Data/test_preprocessed_first.csv')

In [16]:
#test=pd.read_csv('Data/train_preprocessed_first.csv')
#train=pd.read_csv('Data/test_preprocessed_first.csv')

## Lemmatization

In [17]:
comments_train=train['comment_text']
comments_test=test['content']

In [18]:
comments_train=list(comments_train)
comments_test=list(comments_test)

In [19]:
wordnet_lemmatizer = WordNetLemmatizer()

In [20]:
#Lemmatization
#Pos Tagging-Noun,verb,adjective,adverb
def lemma(text, lemmatization=True):
  output=""
  if lemmatization:
    text=text.split(" ")
    for word in text:
       word1 = wordnet_lemmatizer.lemmatize(word, pos = "n")
       word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
       word3 = wordnet_lemmatizer.lemmatize(word2, pos = "a")
       word4 = wordnet_lemmatizer.lemmatize(word3, pos = "r")
       output=output + " " + word4
  else:
    output=text
  
  return str(output.strip()) 

Lemmatizing Training Data

In [21]:
lemmatized_train_data = [] 

for line in tqdm_notebook(comments_train, total=159571): 
    lemmatized_train_data.append(lemma(line))

  0%|          | 0/159571 [00:00<?, ?it/s]

In [22]:
lemmatized_train_data[152458]

'i agree with you about graemels intention he be a revert nazi and wikipedia ha a way of protect against this report him to the wprr threerevertrule notice board post by report the revert nazi graemel ha show himselfherself to be a revert nazi this be unacceptable on a site that be make by it user and not it admins if you feel you have be unjustly revert more than time over a hour period please report himher to the threerevertnoticeboard wprr it be time to take back what be ours'

Lemmatizing Test Data

In [23]:
lemmatized_test_data = [] 

for line in tqdm_notebook(comments_test, total=len(comments_test)): 
    lemmatized_test_data.append(lemma(line))

  0%|          | 0/63812 [00:00<?, ?it/s]

## Stopwords Removal

In [24]:
stopword_list=STOP_WORDS

Adding Single and Dual to STOP_WORDS

In [25]:
def iter_all_strings():
    for size in itertools.count(1):
        for s in itertools.product(ascii_lowercase, repeat=size):
            yield "".join(s)

dual_alpha_list=[]
for s in iter_all_strings():
    dual_alpha_list.append(s)
    if s == 'zz':
        break

In [26]:
dual_alpha_list.remove('i')
dual_alpha_list.remove('a')
dual_alpha_list.remove('am')
dual_alpha_list.remove('an')
dual_alpha_list.remove('as')
dual_alpha_list.remove('at')
dual_alpha_list.remove('be')
dual_alpha_list.remove('by')
dual_alpha_list.remove('do')
dual_alpha_list.remove('go')
dual_alpha_list.remove('he')
dual_alpha_list.remove('hi')
dual_alpha_list.remove('if')
dual_alpha_list.remove('is')
dual_alpha_list.remove('in')
dual_alpha_list.remove('me')
dual_alpha_list.remove('my')
dual_alpha_list.remove('no')
dual_alpha_list.remove('of')
dual_alpha_list.remove('on')
dual_alpha_list.remove('or')
dual_alpha_list.remove('ok')
dual_alpha_list.remove('so')
dual_alpha_list.remove('to')
dual_alpha_list.remove('up')
dual_alpha_list.remove('us')
dual_alpha_list.remove('we')

In [27]:
for letter in dual_alpha_list:
    stopword_list.add(letter)
print("Done!!")

Done!!


Checking for other words that we may need in STOP_WORDS

In [28]:
def search_stopwords(data, search_stop=True):
  output=""
  if search_stop:
    data=data.split(" ")
    for word in data:
      if not word in stopword_list:
        output=output+" "+word 
  else:
    output=data

  return str(output.strip())  

In [29]:
potential_stopwords = [] 

for line in tqdm_notebook(lemmatized_train_data, total=159571): 
    potential_stopwords.append(search_stopwords(line))

  0%|          | 0/159571 [00:00<?, ?it/s]

In [30]:
len(potential_stopwords)

223549

Combining all the sentences in the list into a single string

In [31]:
def string_combine(stopword):
  final_a=""
  for item in range(1,159571):
    final_a=final_a+" "+stopword[item]
  return final_a



In [32]:
total_string_potential=string_combine(potential_stopwords)

In [ ]:
#print(potential_stopwords)

Counting the number of words in each of the 4 strings

In [33]:
def word_count(str):
    counts = dict()
    words = str.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

In [34]:
total_string_potential_dict=word_count(total_string_potential)


Converting Dictionaries to Dataframe

In [35]:
total_string_potential_df = pd.DataFrame(list(total_string_potential_dict.items()),columns = ['Word','Count'])

Getting Dataframe output in descending order

In [36]:
total_string_potential_df

,Word,Count
0,daww,1
1,match,1062
2,background,770
3,colour,335
4,seemingly,165
...,...,...
195350,nearendless,1
195351,imagebarackobamamother,2
195352,hanumakonda,1
195353,automaker,1


Retaining certain words and removing others from the above list

In [37]:
potential_stopwords=['editor', 'reference', 'thank', 'work','find', 'good', 'know', 'like', 'look', 'thing', 'want', 'time', 'list', 'section','wikipedia', 'doe', 'add','new', 'try', 'think', 'write','use', 'user', 'way', 'page']

Adding above retrived words into the stopwords list

In [38]:
for word in potential_stopwords:
    stopword_list.add(word)
print("Done!!")

Done!!


Removing Stopwords from Training Data

In [39]:
def remove_stopwords(text, remove_stop=True):
  output = ""
  if remove_stop:
    text=text.split(" ")
    for word in text:
      if word not in stopword_list:
        output=output + " " + word
  else :
    output=text

  return str(output.strip())      

In [40]:
processed_train_data = [] 

for line in tqdm_notebook(lemmatized_train_data, total=159571): 
    processed_train_data.append(remove_stopwords(line))

  0%|          | 0/159571 [00:00<?, ?it/s]

In [41]:
processed_train_data[152458]

'agree graemels intention revert nazi protect report wprr threerevertrule notice board post report revert nazi graemel himselfherself revert nazi unacceptable site admins feel unjustly revert hour period report himher threerevertnoticeboard wprr'

Removing Stopwords from Test Data

In [42]:
processed_test_data = [] 

for line in tqdm_notebook(lemmatized_test_data, total=153164): 
    processed_test_data.append(remove_stopwords(line))

  0%|          | 0/153164 [00:00<?, ?it/s]

# Model Building

In [43]:
max_features=100000      
maxpadlen = 200          
val_split = 0.2      
embedding_dim_fasttext = 300

Tokenization

In [44]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(processed_train_data))
list_tokenized_train = tokenizer.texts_to_sequences(processed_train_data)
list_tokenized_test = tokenizer.texts_to_sequences(processed_test_data)

In [45]:
word_index=tokenizer.word_index
print("Words in Vocabulary: ",len(word_index))

Words in Vocabulary:  263853


Padding

In [46]:
X_t=pad_sequences(list_tokenized_train, maxlen=maxpadlen, padding = 'post')
X_te=pad_sequences(list_tokenized_test, maxlen=maxpadlen, padding = 'post')

In [47]:
print('Tokenized sentences: \n', X_t[10])
print('One hot label: \n', y[10])

Tokenized sentences: 
 [  148   557 40917   240   197 40917   240    96    17   590    17   148
   359   557     1  1108   148   331  5462   148   151    17   378   270
   359   557    17     1  1343   148    17   378     2    32   148   557
   197   148   181    47    91   590   148   557    17     2   489   130
    10  1095   453  1093    17  2449   503    35   148    17   197   396
   359    11   274   197   227   161   118    21    19    29   181    78
    19     3   428  4765     5 40917   240   197 40917   240    96   271
   378   303    55   590    37    42    78   427  1501    37   271     7
   590  1319    78  1717   137    10   137  4994   137    93    42   519
  1233    16    78  1379   115   137  1689    78  1166     5   504    78
   478    36   271    55  3528   166  2233   466    36   308  1664    66
   181   237   161  2682    36    36  1674    78  5209  1674    78    36
    78    36   197   271    47    91   590     5    36   271   197    77
    10   842  3323    17    

In [48]:
indices = np.arange(X_t.shape[0])
np.random.shuffle(indices)

In [49]:
X_t = X_t[indices]
labels = y[indices]
print('Features are :','\n',X_t)
print('labels are:','\n',labels)

Features are : 
 [[ 232   31   39 ...    0    0    0]
 [9620 3355 1287 ...    0    0    0]
 [ 520  119    8 ...    0    0    0]
 ...
 [ 316 2745  579 ...    0    0    0]
 [ 381  422 2378 ...    0    0    0]
 [4118  405  126 ...    0    0    0]]
labels are: 
 [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 1 0 1 0]
 ...
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 1 0 0 0]]


### Splitting data into Training and Validation Set

In [50]:
num_validation_samples = int(val_split*X_t.shape[0])
x_train = X_t[: -num_validation_samples]
y_train = labels[: -num_validation_samples]
x_val = X_t[-num_validation_samples: ]
y_val = labels[-num_validation_samples: ]


In [51]:
print('Number of entries in each category:')
print('training: ', y_train.sum(axis=0))
print('validation: ', y_val.sum(axis=0))

Number of entries in each category:
training:  [17094  1585  9689   561  9024  1704]
validation:  [4290  377 2451  128 2280  413]


### Importing Fast Text

In [54]:
import requests, zipfile, io

zip_file_url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"

r = requests.get(zip_file_url)

z = zipfile.ZipFile(io.BytesIO(r.content))

z.extractall()


MemoryError: 

In [ ]:
import codecs
embeddings_index_fasttext = {}

f = codecs.open('wiki-news-300d-1M.vec', encoding='utf-8')


for line in f:

    values = line.rstrip().rsplit(' ')

    word = values[0]

    coefs = np.asarray(values[1:], dtype='float32')

    embeddings_index_fasttext[word] = coefs

f.close()


In [54]:
embedding_matrix_fasttext = np.random.random((len(word_index) + 1, embedding_dim_fasttext))
for word, i in word_index.items():
    embedding_vector = embeddings_index_fasttext.get(word)
    if embedding_vector is not None:
        embedding_matrix_fasttext[i] = embedding_vector
        
print("Completed!")
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix_fasttext, axis=1) == 0))

Completed!
number of null word embeddings: 0


### Creating Model

In [182]:
# !pip install keras-bert
# !pip install keras-rectified-adam
#!wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
#!unzip -o uncased_L-12_H-768_A-12.zip

from keras.models import load_model
from keras.layers import concatenate
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps
from keras.layers import Input
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Model, load_model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.layers import Input, Embedding, SpatialDropout1D, Bidirectional, GRU, GlobalMaxPooling1D, Dense

import tensorflow as tf

In [183]:
SEQ_LEN = 128
BATCH_SIZE = 32
EPOCHS = 2
LR = 1e-4
import os
pretrained_path = r'D:\Aaryan project\Toxic-comment\uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
bert_config_file=r'D:\Aaryan project\Toxic-comment\uncased_L-12_H-768_A-12\bert_config.json'
#bert_ckpt_file=r'C:\Users\RD\Toxic-Comment-Classifier-using-Deep-Learning\uncased_L-12_H-768_A-12\bert_config.json'
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [184]:
inp=Input(shape=(maxpadlen, ),dtype='int32')
def build_model():
    inp=Input(shape=(maxpadlen, ),dtype='int32')
    from tensorflow.keras.layers import BatchNormalization
#     from tensorflow.keras.layers import Input,Dense,Lambda
    model = load_trained_model_from_checkpoint(
        config_path,
        checkpoint_path,
        training=True,
        trainable=True,
        seq_len=SEQ_LEN,
    )
#     inp=Input(shape=(maxpadlen, ),dtype='int32')
    inputs = model.inputs[:2]
    dense = model.layers[-3].output
    dense2 = keras.layers.Dense(10,activation='relu', kernel_initializer ='glorot_uniform')(dense)
    dense3 = keras.layers.Dense(10,activation='relu', kernel_initializer ='glorot_uniform')(dense2)
    outputs = keras.layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_uniform',
                                 name = 'real_output')(dense3)

    
   
    model = keras.models.Model(inputs,outputs)
       
    return model


In [3]:
model_bert = build_model()
model_bert.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_bert.summary()

NameError: name 'build_model' is not defined

In [152]:
# import tensorflow as tf

# from bert import BertModelLayer
# from bert.loader import StockBertConfig, load_stock_weights
# from bert.loader import map_to_stock_variable_name

# model_bert = build_model()





In [153]:
# model_bert.summary()

Model: "model_37"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input-Token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 Input-Segment (InputLayer)     [(None, 128)]        0           []                               
                                                                                                  
 Embedding-Token (TokenEmbeddin  [(None, 128, 768),  23440896    ['Input-Token[0][0]']            
 g)                              (30522, 768)]                                                    
                                                                                                  
 Embedding-Segment (Embedding)  (None, 128, 768)     1536        ['Input-Segment[0][0]']   

                                                                                                  
 Encoder-3-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-2-FeedForward-Norm[0][0
 on-Add (Add)                                                    ]',                              
                                                                  'Encoder-3-MultiHeadSelfAttentio
                                                                 n-Dropout[0][0]']                
                                                                                                  
 Encoder-3-MultiHeadSelfAttenti  (None, 128, 768)    1536        ['Encoder-3-MultiHeadSelfAttentio
 on-Norm (LayerNormalization)                                    n-Add[0][0]']                    
                                                                                                  
 Encoder-3-FeedForward (FeedFor  (None, 128, 768)    4722432     ['Encoder-3-MultiHeadSelfAttentio
 ward)    

 on-Dropout (Dropout)                                            n[0][0]']                        
                                                                                                  
 Encoder-6-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-5-FeedForward-Norm[0][0
 on-Add (Add)                                                    ]',                              
                                                                  'Encoder-6-MultiHeadSelfAttentio
                                                                 n-Dropout[0][0]']                
                                                                                                  
 Encoder-6-MultiHeadSelfAttenti  (None, 128, 768)    1536        ['Encoder-6-MultiHeadSelfAttentio
 on-Norm (LayerNormalization)                                    n-Add[0][0]']                    
                                                                                                  
 Encoder-6

 Encoder-9-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-9-MultiHeadSelfAttentio
 on-Dropout (Dropout)                                            n[0][0]']                        
                                                                                                  
 Encoder-9-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-8-FeedForward-Norm[0][0
 on-Add (Add)                                                    ]',                              
                                                                  'Encoder-9-MultiHeadSelfAttentio
                                                                 n-Dropout[0][0]']                
                                                                                                  
 Encoder-9-MultiHeadSelfAttenti  (None, 128, 768)    1536        ['Encoder-9-MultiHeadSelfAttentio
 on-Norm (LayerNormalization)                                    n-Add[0][0]']                    
          

                                                                                                  
 Encoder-12-MultiHeadSelfAttent  (None, 128, 768)    0           ['Encoder-12-MultiHeadSelfAttenti
 ion-Dropout (Dropout)                                           on[0][0]']                       
                                                                                                  
 Encoder-12-MultiHeadSelfAttent  (None, 128, 768)    0           ['Encoder-11-FeedForward-Norm[0][
 ion-Add (Add)                                                   0]',                             
                                                                  'Encoder-12-MultiHeadSelfAttenti
                                                                 on-Dropout[0][0]']               
                                                                                                  
 Encoder-12-MultiHeadSelfAttent  (None, 128, 768)    1536        ['Encoder-12-MultiHeadSelfAttenti
 ion-Norm 

#### Training Model with Best Parameters

BERT

In [186]:
inp=Input(shape=(maxpadlen, ),dtype='int32')
embedding_layer = Embedding(len(word_index) + 1,
                           embedding_dim_fasttext,
                           weights = [embedding_matrix_fasttext],
                           input_length = maxpadlen,
                           trainable=False,
                           name = 'embeddings')
embedded_sequences = embedding_layer(inp)

In [195]:
model_bert.summary()

Model: "model_47"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input-Token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 Input-Segment (InputLayer)     [(None, 128)]        0           []                               
                                                                                                  
 Embedding-Token (TokenEmbeddin  [(None, 128, 768),  23440896    ['Input-Token[0][0]']            
 g)                              (30522, 768)]                                                    
                                                                                                  
 Embedding-Segment (Embedding)  (None, 128, 768)     1536        ['Input-Segment[0][0]']   

                                                                                                  
 Encoder-3-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-2-FeedForward-Norm[0][0
 on-Add (Add)                                                    ]',                              
                                                                  'Encoder-3-MultiHeadSelfAttentio
                                                                 n-Dropout[0][0]']                
                                                                                                  
 Encoder-3-MultiHeadSelfAttenti  (None, 128, 768)    1536        ['Encoder-3-MultiHeadSelfAttentio
 on-Norm (LayerNormalization)                                    n-Add[0][0]']                    
                                                                                                  
 Encoder-3-FeedForward (FeedFor  (None, 128, 768)    4722432     ['Encoder-3-MultiHeadSelfAttentio
 ward)    

 on-Dropout (Dropout)                                            n[0][0]']                        
                                                                                                  
 Encoder-6-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-5-FeedForward-Norm[0][0
 on-Add (Add)                                                    ]',                              
                                                                  'Encoder-6-MultiHeadSelfAttentio
                                                                 n-Dropout[0][0]']                
                                                                                                  
 Encoder-6-MultiHeadSelfAttenti  (None, 128, 768)    1536        ['Encoder-6-MultiHeadSelfAttentio
 on-Norm (LayerNormalization)                                    n-Add[0][0]']                    
                                                                                                  
 Encoder-6

 Encoder-9-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-9-MultiHeadSelfAttentio
 on-Dropout (Dropout)                                            n[0][0]']                        
                                                                                                  
 Encoder-9-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-8-FeedForward-Norm[0][0
 on-Add (Add)                                                    ]',                              
                                                                  'Encoder-9-MultiHeadSelfAttentio
                                                                 n-Dropout[0][0]']                
                                                                                                  
 Encoder-9-MultiHeadSelfAttenti  (None, 128, 768)    1536        ['Encoder-9-MultiHeadSelfAttentio
 on-Norm (LayerNormalization)                                    n-Add[0][0]']                    
          

                                                                                                  
 Encoder-12-MultiHeadSelfAttent  (None, 128, 768)    0           ['Encoder-12-MultiHeadSelfAttenti
 ion-Dropout (Dropout)                                           on[0][0]']                       
                                                                                                  
 Encoder-12-MultiHeadSelfAttent  (None, 128, 768)    0           ['Encoder-11-FeedForward-Norm[0][
 ion-Add (Add)                                                   0]',                             
                                                                  'Encoder-12-MultiHeadSelfAttenti
                                                                 on-Dropout[0][0]']               
                                                                                                  
 Encoder-12-MultiHeadSelfAttent  (None, 128, 768)    1536        ['Encoder-12-MultiHeadSelfAttenti
 ion-Norm 

In [2]:
# model_info_1=model_bert.fit(x_train,y_train, epochs=2, batch_size=32,  validation_data=(x_val, y_val))
model_info_1=model_bert.fit(x_train,y_train, epochs=2, batch_size=32,  validation_data=(x_val, y_val))


NameError: name 'model_bert' is not defined

## Plotting Graphs

### GRU

In [ ]:
loss = model_info_1.history['loss']
val_loss = model_info_1.history['val_loss']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show();

In [ ]:
accuracy = model_info_1.history['accuracy']
val_accuracy = model_info_1.history['val_accuracy']

plt.plot(epochs, accuracy, label='Training accuracy')
plt.plot(epochs, val_accuracy, label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.show();

# Saving the Model

In [ ]:
model_bert.save('Data/model_BERT')

INFO:tensorflow:Assets written to: Data/model_BERT\assets


# Loading Saved Model

In [ ]:
loaded_model_bert = keras.models.load_model('Data/model_BERT')

# Generating the Output

### BERT

In [ ]:
test_values_1 = loaded_model_bert.predict([X_te], batch_size=1, verbose=1)

153164/153164 [==============================] - 39546s 258ms/step


In [ ]:
sample_submission = pd.read_csv(r'C:\Users\RD\Toxic-Comment-Classifier-using-Deep-Learning\Data\sample_submission.csv')
test_values_1=pd.DataFrame(test_values_1,columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
submission = pd.DataFrame(sample_submission["id"])
combined_submission=pd.concat([submission,test_values_1],axis=1)
combined_submission.to_csv(r'C:\Users\RD\Toxic-Comment-Classifier-using-Deep-Learning\Data\combined_submission_bert_model.csv', index=False)

# Testing the Created Model

In [ ]:
def toxicity_level(string):
    new_string = [string]
    new_string = tokenizer.texts_to_sequences(new_string)
    new_string = pad_sequences(new_string, maxlen=maxpadlen, padding='post')
    
    prediction = loaded_model_bert.predict(new_string) 
    
    print("Toxicity levels for '{}':".format(string))
    print('Toxic:         {:.0%}'.format(prediction[0][0]))
    print('Severe Toxic:  {:.0%}'.format(prediction[0][1]))
    print('Obscene:       {:.0%}'.format(prediction[0][2]))
    print('Threat:        {:.0%}'.format(prediction[0][3]))
    print('Insult:        {:.0%}'.format(prediction[0][4]))
    print('Identity Hate: {:.0%}'.format(prediction[0][5]))
    print()
    
    return

In [ ]:
toxicity_level('go jump off a bridge jerk')

Toxicity levels for 'go jump off a bridge jerk':
Toxic:         89%
Severe Toxic:  2%
Obscene:       39%
Threat:        0%
Insult:        51%
Identity Hate: 0%



In [ ]:
toxicity_level('i will kill you')

Toxicity levels for 'i will kill you':
Toxic:         32%
Severe Toxic:  1%
Obscene:       3%
Threat:        16%
Insult:        2%
Identity Hate: 0%



In [ ]:
toxicity_level('have a nice day')

Toxicity levels for 'have a nice day':
Toxic:         1%
Severe Toxic:  0%
Obscene:       0%
Threat:        0%
Insult:        0%
Identity Hate: 0%



In [ ]:
toxicity_level('fuck ofF!!')

Toxicity levels for 'fuck ofF!!':
Toxic:         99%
Severe Toxic:  41%
Obscene:       100%
Threat:        0%
Insult:        60%
Identity Hate: 1%



In [ ]:
toxicity_level('Hello, How are you?')

Toxicity levels for 'Hello, How are you?':
Toxic:         1%
Severe Toxic:  0%
Obscene:       1%
Threat:        0%
Insult:        0%
Identity Hate: 0%



In [ ]:
toxicity_level('get the fuck away from me @sshole!!')

Toxicity levels for 'get the fuck away from me @sshole!!':
Toxic:         99%
Severe Toxic:  35%
Obscene:       99%
Threat:        1%
Insult:        59%
Identity Hate: 1%

